V4.1

18-2-2025

In [ ]:
# 1

import re
import pandas as pd
import os
from google.colab import drive
import csv
import shutil
from posixpath import curdir

In [ ]:
# 2

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

folder_to_read_from="/content/drive/MyDrive/تـحـويـل تـصـديـرات (File responses)/work directory/latest chats/"
folder_to_write_to="/content/drive/MyDrive/تـحـويـل تـصـديـرات (File responses)/work directory/result/"
upload_folder="/content/drive/MyDrive/تـحـويـل تـصـديـرات (File responses)/step 1: upload files or access folder directly/"
rosetta_path="/content/drive/MyDrive/تـحـويـل تـصـديـرات (File responses)/work directory/latest chats/rosetta.csv"
Dictionary_path="/content/drive/MyDrive/تـحـويـل تـصـديـرات (File responses)/work directory/result/Dictionary.csv"

if not os.path.isfile(rosetta_path):
    with open(rosetta_path, 'w', newline='') as file:
        csvwriter = csv.writer(file)
        csvwriter.writerow(['chat', 'name', 'restaurant'])

cur=os.listdir(upload_folder)
feu=os.listdir(folder_to_read_from)
for bn in cur:
  if bn in feu:
    os.remove(folder_to_read_from+bn)
    shutil.move(upload_folder+bn, folder_to_read_from+bn)
  else:
    shutil.move(upload_folder+bn, folder_to_read_from+bn)

In [ ]:
# 3 v4 (working)

def convert(read, write, smsr, rosetta):

  original_source = open(read+smsr,"r")
  content = original_source.readlines()
  original_source.close()

  newsa = []


  for q in content:

          q=q.replace("\n"," ")
          q=q.replace("\u202f","")

          if re.match("\d{2}/\d{2}/\d{4}",q):
              q=q.strip()
              newsa.append(q)
          else:
              q=q.strip()
              newsa.append(newsa.pop() + " " + q)
###     understand local and global variables before moving these lists
  date=[]
  time=[]
  datetime=[]
  author=[]
  texts=[]
  blabla=[]
  booty=[]
  not_booty=[]
  unique_senders=[]
  triggers=["added",
            "created",
            "changed",
            "removed",
            "pinned",
            "left",
            "was added",
            "were added"
            "were",
            "was"]
##########################################################
  qi=0
  for message in newsa:

    try:
      sender=message.split(":")[1]
      sender=sender.split("-")[1]
      sender=sender.strip()

    except IndexError:
      del newsa[qi]
    else:
      if not any(x in sender for x in triggers):
          if not sender in unique_senders:
              unique_senders.append(sender)
    qi+=1
##########################################################
  qi=0
  for message in newsa:
    try:
      sender=message.split(":")[1]
      sender=sender.split("-")[1]
      sender=sender.strip()
    except IndexError:
      del newsa[qi]
    else:
      if not sender in unique_senders:
              booty.append(sender)
    qi+=1
##########################################################
  for damned in booty:
      for x in triggers:
        if not damned.find(x)==-1:
          nb=re.split(x,damned)[0].strip()
          if not nb.find("was")==-1:
            bn = re.split("was",nb)[0].strip()
            unique_senders.append(bn)
          else: unique_senders.append(nb)
##########################################################
  qi=0
  for message in newsa:

      timedate=message.split("-")[0]
      timedate=timedate.replace(",","")
      timedate=timedate.strip()
      date = timedate[:11]
      time = ''.join([char for char in timedate[11:] if char != ' '])
      timedate = date+time
      datetime.append(timedate)
      quote=message.split("-")[1:]

      if len(quote)>1:
          quote="-".join(quote)
          quote=quote.strip()
          blabla.append(quote)

      else:
        try:
          quote[0]=quote[0].strip()
          blabla.append(quote[0])
        except IndexError:
          del newsa[qi]
      qi+=1

      for i in unique_senders:
          if blabla[-1].startswith(i):
              author.append(i)
              break
          elif blabla[-1].startswith(i+':'):
              author.append(i)
              break
##########################################################
  for n in blabla:
      for i in unique_senders:
              if n.startswith(i+':'):
                  n=n.split(i+':',1)[1]
      texts.append(n.strip())
##########################################################
# move?
  for i in datetime:
      i=i.replace(",","")
      # date
##########################################################
  with open(rosetta, 'r', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    data = list(csvreader)

    string_in_first_column = any(row[0] == smsr for row in data)


    if not string_in_first_column:
        next_value = len(data) - 1
        data.append([smsr, "", next_value])


    with open(rosetta, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(data)
#######################################################
  def get_value_from_row(rosetta, smsr):
    with open(rosetta, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if row and row[0] == smsr:
                value = row[2]
                return value
    return None

  value = get_value_from_row(rosetta, smsr)
##########################################################
  df = pd.DataFrame({"Datetime": datetime, "Author": author, "Message": texts, "Value": value})


  df['Datetime']=pd.to_datetime(df['Datetime'], format="%d/%m/%Y %I:%M%p")



  df.to_csv(write+smsr[:-4]+'.csv', index=False, encoding='utf-8-sig')



In [ ]:
# 4

def extract_columns(file_path, new_file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        headers = next(csvreader)
        column_indices = [1, 2]

        with open(new_file_path, 'w', newline='', encoding='utf-8') as new_csvfile:
            csvwriter = csv.writer(new_csvfile)

            new_headers = [headers[i] for i in column_indices]
            csvwriter.writerow(new_headers)

            for row in csvreader:
                new_row = [row[i] for i in column_indices]
                csvwriter.writerow(new_row)

def remove_repeated_suffix(old_file_name):
    pattern = re.compile(r'\s?\(\d+\)\.txt$')
    match = pattern.search(old_file_name)
    if match:
        new_file_name = pattern.sub('.txt', old_file_name)
    else:
        new_file_name = old_file_name
    return new_file_name

file_names=os.listdir(folder_to_read_from)

opr=[]

for old_file_name in file_names:
  if not old_file_name == "rosetta.csv":
    if old_file_name.endswith(".txt"):
      new_file_name=remove_repeated_suffix(old_file_name)
      os.rename(folder_to_read_from+old_file_name, folder_to_read_from+new_file_name)
      opr.append(new_file_name)


for smsr in opr:
  convert(folder_to_read_from, folder_to_write_to, smsr, rosetta_path)

extract_columns(rosetta_path, Dictionary_path)